# Module 1: Introduction to Exploratory Analysis 

What we'll be doing in this notebook:
-----

 1.  Checking variable type
 2.  Checking for missing variables 
 3.  Number of observations in the dataset
 4.  Descriptive statistics

### Import packages

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser

# The command below means that the output of multiple commands in a cell will be output at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

# Show figures in notebook
%matplotlib inline

### Import dataset

We read in our dataset

In [16]:
path = '../data/'
filename = 'loans.csv'

try:
    df = pd.read_csv(path+filename)
except FileNotFoundError:
    # If data is not found, download it from GitHub
    import os
    os.system(f'git clone --single-branch --depth=1 https://github.com/DeltaAnalytics/machine_learning_for_good_data {path}')
    df = pd.read_csv(path+filename)

In the cell below, we take a random sample of 2 rows to get a feel for the data.

In [17]:
df.sample(n=18)

,id_number,loan_amount,lender_count,status,funded_date,funded_amount,repayment_term,location_country_code,sector,description,use
3897,1566972,325,13,funded,2018-07-22T18:47:47Z,325,11,SL,Food,This is 49-year-old Wulamatu from Port Loko br...,"to purchase more rice, fish, palm oil, onions,..."
4168,616482,500,19,funded,2013-10-22T19:57:50Z,500,8,SO,Education,Sahra Mahamoud is 23 years old. She has two ch...,to buy a laptop for education purposes.
3054,1569662,275,10,funded,2018-07-19T15:47:04Z,275,18,MZ,Housing,Clara is 31 years old and is a single mother o...,to buy construction materials.
1912,1571561,300,0,fundraising,NaN,0,14,KE,Agriculture,"Meet Daniel, a hardworking farmer who lives wi...",to pay for farm inputs such as quality fertili...
1476,1567846,850,5,fundraising,NaN,125,11,GH,Clothing,"Elizabeth lives in Hohoe, in the Volta Region ...",to buy stock of used clothing and bedsheets fo...
5402,1520210,4475,102,funded,2018-05-07T14:00:36Z,4475,6,CD,Food,Ms. Charlotte is a client of the MFI Hekima wh...,to buy ten baskets of fish to sell and grow he...
5713,1567837,625,23,funded,2018-07-21T10:12:31Z,625,7,TZ,Transportation,Abdul is 40 years old and is a married father ...,"to decorate his bajaj, change seat covers and ..."
2899,1507382,2675,13,funded,2018-04-19T18:39:53Z,2675,8,ML,Agriculture,Ladji Ousmane raises livestock and is the one ...,to buy animals to resell.
775,1521300,475,5,funded,2018-05-07T03:32:33Z,475,20,CM,Agriculture,The borrower is married and a father of five c...,"to construct a pig sty and buy piglets, goats,..."
2793,1518314,1875,74,funded,2018-05-01T17:59:16Z,1875,6,MW,Food,Lucia is 39 years old. She is married with fiv...,to pay for more fish and timber.


### 1) Type Checking
<a id='type_check'></a>

Type is very important in Python programing, because it affects the types of functions you can apply to a series. There are a few different types of data you will see regularly (see [this](https://en.wikibooks.org/wiki/Python_Programming/Data_Types) link for more detail):
* **int** - a number with no decimal places. example: loan_amount field
* **float** - a number with decimal places. example: partner_id field
* **str** - str is short for string. This type formally defined as a sequence of unicode characters. More simply, string means that the data is treated as a word, not a number. example: sector
* **boolean** - can only be True or False. There is not currently an example in the data, but we will be creating a gender field shortly.
* **datetime** - values meant to hold time data. Example: posted_date

Let's check the type of our variables using the examples we saw in the cell above.

In [18]:
# Here are all of the columns
df.columns.tolist()

['id_number',
 'loan_amount',
 'lender_count',
 'status',
 'funded_date',
 'funded_amount',
 'repayment_term',
 'location_country_code',
 'sector',
 'description',
 'use']

In [19]:
# Find the dtype, aka datatype, for a column
df['id_number'].dtype

dtype('int64')

In [20]:
# Try this - Pick a couple of columns and check their type on your own
for pr in df.columns.tolist():
    print( pr , " " , df[pr].dtype)

id_number   int64
loan_amount   int64
lender_count   int64
status   object
funded_date   object
funded_amount   int64
repayment_term   int64
location_country_code   object
sector   object
description   object
use   object


### 2) Do I have missing values?

<a id='missing_check'></a>

If we have missing data, is the missing data at random or not? If data is missing at random, the data distribution is still representative of the population. You can probably ignore the missing values as an inconvenience. However, if the data is systematically missing, the analysis you do may be biased. You should carefully consider the best way to clean the data, it may involve dropping some data.

We want to see how many values are missing in certain variable columns. One way to do this is to count the number of null observations. 

For this, we wrote a short function to apply to the dataframe. 

We print out the first few observations, but you can remove the .head() to print out all columns. 

In [21]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
## Check how many are missing by column, and then check which ones have any missing values
print(df.apply(num_missing, axis=0).where(lambda x : x != 0).dropna()) 
#axis=0 defines that function is to be applied on each column

Missing values per column:
funded_date              937.0
location_country_code     17.0
description              342.0
use                      342.0
dtype: float64


In [22]:
data = df.drop(columns=['funded_date', 'description' , 'use'])
data.head(20)

,id_number,loan_amount,lender_count,status,funded_amount,repayment_term,location_country_code,sector
0,736066,4825,60,funded,4825,8,BJ,Retail
1,743090,975,34,funded,975,12,BJ,Food
2,743120,950,25,funded,950,14,BJ,Services
3,743121,825,28,funded,825,14,BJ,Retail
4,743124,725,21,funded,725,13,BJ,Retail
5,743125,725,27,funded,725,13,BJ,Retail
6,743130,725,27,funded,725,13,BJ,Retail
7,743134,525,14,funded,525,13,BJ,Services
8,743207,950,35,funded,950,14,BJ,Retail
9,743228,500,10,funded,500,10,BJ,Food


### 3) Sanity Checks
<a id='obs_check'></a>

**Does the dataset match what you expected to find?**
- is the range of values what you would expect. For example, are all loan_amounts above 0.
- do you have the number of rows you would expect
- is your data for the date range you would expect. For example, is there a strange year in the data like 1880.
- are there unexpected spikes when you plot the data over time


In the command below we find out the number of loans and number of columns by using the function shape. You can also use len(df.index) to find the number of rows.

In [23]:
print(f'There are {df.shape[0]} observations and {df.shape[1]} features')

There are 6019 observations and 11 features


Remember, each row is an observation and each column is a potential feature. 

Remember we need large about of data for machine learning.

### 4) Descriptive statistics of the dataset

<a id='desc_stats'></a>

The "describe" command below provides key summary statistics for each numeric column.

In [24]:
df.describe()

,id_number,loan_amount,lender_count,funded_amount,repayment_term
count,6.019000e+03,6019.000000,6019.000000,6019.000000,6019.000000
mean,1.359770e+06,1499.011464,35.661406,1325.070610,11.803290
std,3.719316e+05,2512.517280,73.420256,2444.726815,9.114948
min,1.377200e+04,50.000000,0.000000,0.000000,3.000000
25%,1.425188e+06,300.000000,7.000000,200.000000,8.000000
50%,1.550673e+06,625.000000,16.000000,525.000000,10.000000
75%,1.566204e+06,1825.000000,41.000000,1525.000000,14.000000
max,1.573593e+06,80000.000000,2665.000000,80000.000000,133.000000


In order to get the same summary statistics for categorical columns (string) we need to do a little data wrangling. The first line of code filters for all columns that are a data type object. As we know from before this means they are considered to be a string. The final row of code provides summary statistics for these character fields.

In [25]:
categorical = df.dtypes[df.dtypes == "object"].index
df[categorical].describe()

,status,funded_date,location_country_code,sector,description,use
count,6019,5082,6002,6019,5677,5677
unique,3,4453,30,14,5277,4325
top,funded,2018-07-22T15:54:41Z,CD,Food,Mr. Bishweka is a teacher in a local school. D...,to pay for a stove.
freq,5082,9,400,1738,2,80


In the table above, there are 4 really useful fields: 

1) **count** - total number of fields populated (Not empty). 

2) **unique** - tells us how many different unique ways this field is populated. For example 4 in description.languages tells us there are 4 different language descriptions. 

3) **top** - tells us the most popular data point. For example, the top activity in this dataset is Farming which tells us most loans are in Farming.

4) **freq** - tells us that how frequent the most popular category is in our dataset. For example, 'en' (English) is the language almost all descriptions (description.languages) are written in (118,306 out of 118,316).

What is next
-----

In the next section, we move on to exploratory data analysis (EDA).

<br>
<br> 
<br>

----